<a href="https://colab.research.google.com/github/rezendervp/chemical-engineering/blob/main/SPH_2D_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from re import I
"""
Simulates the free-surface flow of "pouring a beer" using Smoothed Particle
Hydrodynamics (SPH). Uses the super-simple approch of Matthias Müller:
https://matthias-research.github.io/pages/publications/sca03.pdf

Simulates the Navier-Stokes Momentum in Lagrangian Form

    ρ Du/Dt = − ∇p + μ ∇²u + g

u : velocity
ρ : density
p : pressure
μ : dynamics viscosity
g : gravity

Du/Dt : Lagrangian temporal derivative (=Material Derivative)
∇p    : Pressure Gradient
∇²u   : Velocity Laplacian (=collection of second derivatives)

IMPORTANT: The simulated fluid is not incompressible.

-------

Scenario

        +-------------------------+
        |      /  /  /            |
        |     /  /  /             |
        |    ↙  ↙  ↙              |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        |                         |
        +-------------------------+

-> A vertical Rectangular domain with all walls

-> New Particles enter the domain slightly below the top with a velocity
directed to the bottom left (like pouring in the beer)

----------

Solution Strategy:

Discretize the fluid by N particles (i=0, 1, ..., N-1) that smooth
their properties radially with some smoothing kernels. Then the Momentum
Equation discretizes to

Duᵢ/Dt = Pᵢ + Vᵢ + G

with

uᵢ : The velocity of the i-th smoothed particle
Pᵢ : The pressure forces acting on the i-th smoothed particle
Vᵢ : The viscosity forces acting on the i-th smoothed particle
G  : The gravity forces (acting equally on all particles)

This yields a set of N ODEs each for the two velocity components (in case
of 2D) of the particles. These can be solved using a (simpletic) integrator
to advance the position of the particles.


Let xᵢ be the 2D position of each smoothed particle.

Let L be the smoothing length of each smoothed particle.

Let M be the mass of each smoothed particle.

------

Algorithm

(for details on the chosen smoothing kernels, see the paper mentioned above)

1. Compute the rhs for each particle Fᵢ

    1.1 Compute the distances between all particle positions

        dᵢⱼ = || xᵢ − xⱼ ||₂

    1.2 Compute the density at each particle's position (Kernel W_poly6)

        ρᵢ = (315 M) / (64 π L⁹) ∑ⱼ (L² − dᵢⱼ²)³

    1.3 Compute the pressure at each particle's position (κ is the isentropic
        exponent, ρ₀ is a base density)

        pᵢ = κ * (ρ − ρ₀)

    1.4 Compute the pressure force of each particle

        Pᵢ = (− (45 M) / (π L⁶)) ∑ⱼ − (xⱼ − xᵢ) / (dᵢⱼ) (pⱼ + pᵢ) / (2 ρⱼ) (L − dᵢⱼ)²

    1.5 Compute the viscosity force of each particle

        Vᵢ = (45 μ M) / (π L⁶) ∑ⱼ (uⱼ − uᵢ) / (ρⱼ) (L − dᵢⱼ)

    1.6 Add up the RHS

        Fᵢ = Pᵢ + Vᵢ + G

2. Integrate the Ordinary Differential Equation  "ρ Duᵢ/Dt = Fᵢ" with a
   Δt timestep

    2.1 Update the particles' velocities

        uᵢ ← uᵢ + Δt Fᵢ / ρᵢ

    2.2 Update the particles' positions

        xᵢ ← xᵢ + Δt uᵢ

3. Enforce the wall Boundary Conditions. If a particle leaves the
   domain then:

    3.1 Set its position to the Boundary

    3.2 Inverse its velocity component perpendicular to the wall

    3.3 Multiply the velocity component perpendicular to the
        wall with a damping factor

-------

Computational Considerations.

1. The steps on computing density, pressure force and viscosity force
   involve the computation of the various smoothing kernels. Those
   always consist of a constant part that is due to the normalization
   which can be precomputed.

2. When applying summations in the distance calculations and when
   applying the smothing kernels in density, pressure force and
   viscosity force calculations only OTHER PARTICLES IN THE
   SMOOTHING LENGTH OF THE CONSIDERED PARTICLE ARE RELEVANT. Hence,
   we can use efficient neighbor computation routines.

-------

Take care that the ODE integration can become instable when using too
large time steps.
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors
from tqdm import tqdm

Max_particles = 125
Domain_width = 60
Domain_height = 100

Particle_mass = 1
Isotropic_exponent = 20
Base_density = 1
Smoothing_lenght = 5
Dynamic_viscosity = 0.5
Damping_coefficient = - 0.9
Constant_force = np.array([[0.0, -0.1]])  #(2D vector gravity force)

Time_step_lenght = 0.01
N_time_steps = 2_500
Add_particles_every = 50

Figure_size = (4, 6)
Plot_every = 6
Scatter_dot_size =1200


Domain_X_lim = np.array([
    Smoothing_lenght,
    Domain_width - Smoothing_lenght,
])

Domain_Y_lim = np.array([
    Smoothing_lenght,
    Domain_height - Smoothing_lenght,
])

# Normalization Kernel Factors of SPH method

Normalization_density = (
    (
        315 * Particle_mass
    ) / (
        64 * np.pi * Smoothing_lenght**9
    )
)

Normalization_pressure_force = (
    -
    (
        45 * Particle_mass
    ) / (
        np.pi * Smoothing_lenght**6
    )
)

Normalization_viscous_force = (
    (
        45 * Dynamic_viscosity * Particle_mass
    ) / (
        np.pi * Smoothing_lenght**6
    )
)

# Calculations
def main():

    #initial time value
    Time = 0.0
    n_particles = 1

    positions = np.zeros ((n_particles, 2))
    velocities = np.zeros_like(positions)
    forces = np.zeros_like(positions)

    #defining plot style
    plt.style.use("dark_background")

    for iter in tqdm(range(N_time_steps)):
        Time += Time_step_lenght
        if iter % Add_particles_every == 0 and n_particles < Max_particles:
            new_positions = np.array([
                [15 + np.random.rand(), Domain_Y_lim[1]],
                [30 + np.random.rand(), Domain_Y_lim[1]],
                [45 + np.random.rand(), Domain_Y_lim[1]],
            ])

           # new_velocities = np.array([
           #     [-3.0, -15.0],
           #     [ 0.0, -16.0],
           #     [ 3.0, -15.0],
           # ])

            #Random seed velocities
            new_velocities = np.array([
                [-3 + 10*np.random.rand(),  -15 + abs(np.random.rand())],
                [10*np.random.rand(), - 16*  abs(np.random.rand())],
                [3 + 10*np.random.rand(), - 15 + abs(np.random.rand())],
            ])


            #N Particles increment # in more 3 particles at each step
            n_particles += 3

            positions = np.concatenate((positions, new_positions), axis = 0 )
            velocities = np.concatenate((velocities, new_velocities), axis = 0 )

            #1.1)Distances
        neighbors_ids, distances = neighbors.KDTree(
            positions,
        ).query_radius(
            positions,
            Smoothing_lenght,
            return_distance = True,
            sort_results = True,
        )
            #1.2)Density

        densities = np.zeros(n_particles)

        for i in range(n_particles):
            for j_in_list, j in enumerate(neighbors_ids[i]):
                densities[i] += Normalization_density * (
                    Smoothing_lenght**2
                    -
                    distances[i][j_in_list]**2
                )**3

                  #1.3)Pressure

        pressures = Isotropic_exponent * (densities - Base_density)

        #===========================================================
        #Forces Calculation
        # Each parcel is summed at each code block: force += ...

        forces = np.zeros_like(positions)

        #Drop the element itself ( distance zero!)

        neighbors_ids = [ np.delete(x, 0) for x in neighbors_ids]
        distances = [ np.delete(x, 0) for x in distances]

        for i in range(n_particles):
            for j_in_list, j in enumerate(neighbors_ids[i]):
                #1.4) Pressure Forces:
                forces[i] += Normalization_pressure_force* (
                    -
                    (
                        positions[j]
                          -
                        positions[i]
                    ) / distances[i][j_in_list]
                    *
                    (
                        pressures[j]
                        +
                        pressures[i]
                    )  / (2 * densities[j])
                    *
                    (
                        Smoothing_lenght
                        -
                        distances[i][j_in_list]
                    )**2
                )

                #1.5)Viscous Force

                forces[i] += Normalization_viscous_force * (
                    (
                        velocities[j]
                        -
                        velocities[i]
                    ) / densities[j]
                    *
                    (
                        Smoothing_lenght
                        -
                        distances[i][j_in_list]
                    )
                )
        #==============================================================
        #1.6)Gravity Force

        forces += Constant_force * densities[:, np.newaxis] #Total Force P+V+G

        #2)Euler Step Integrator 1st order

        velocities = velocities + Time_step_lenght * forces / densities[:, np.newaxis]
        #density is a 1D array and force is a 2D array. It must be correct the shape
        positions = positions + Time_step_lenght * velocities

        #Enforce Boundary Conditions  (Bololean Auxiliarvariables)

        out_of_left_boundary = positions[:, 0] < Domain_X_lim[0]
        out_of_right_boundary = positions[:, 0] > Domain_X_lim[1]
        out_of_bottom_boundary = positions[:, 1] < Domain_Y_lim[0]
        out_of_top_boundary = positions[:, 1] > Domain_Y_lim[1]

        velocities[out_of_left_boundary, 0]     *= Damping_coefficient
        positions [out_of_left_boundary, 0]      = Domain_X_lim[0]

        velocities[out_of_right_boundary, 0]    *= Damping_coefficient
        positions [out_of_right_boundary, 0]     = Domain_X_lim[1]

        velocities[out_of_bottom_boundary, 1]   *= Damping_coefficient
        positions [out_of_bottom_boundary, 1]    = Domain_Y_lim[0]

        velocities[out_of_top_boundary, 1]      *= Damping_coefficient
        positions [out_of_top_boundary, 1]       = Domain_Y_lim[1]

       #Show Partiles!

        if iter % Plot_every == 0:


          plt.figure(figsize =Figure_size, dpi = 200)
          plt.scatter(
              positions[:, 0],
              positions[:, 1],
              s = Scatter_dot_size,
              c = positions[:, 1],
              cmap = "jet_r",
          )
          title = "Time = {:.2f} [s]".format(Time)
          plt.title(title)
          plt.colorbar()
          plt.ylim(Domain_Y_lim)
          plt.xlim(Domain_X_lim)
          plt.xticks([],[])
          plt.yticks([],[])
          plt.savefig(f'/content/drive/MyDrive/Colab Notebooks/SPH Figures/SPH{iter}.jpg', format='jpg')
          #plt.tight_layout()
          #plt.draw()
          #plt.pause(0.001)
          #plt.clf()
          plt.close("all")


if __name__=="__main__":
  main()

